## Implement change detection pre and post disaster

Note: Based on the sar_deforestation Descartes example

In [34]:
# Import packages
import IPython
import ipywidgets
import ipyleaflet
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
import random
import os
import tensorflow as tf
from utils import FullArray, DrawControl
from unet import UNet
import geopandas as gpd
import descarteslabs as dl
import descarteslabs.workflows as wf
from descarteslabs.vectors import FeatureCollection, properties as p



## Sentinel-1 log ratio

Here, we generate Sentinel-1 SAR median composites for August-September and October-November, and create a log-ratio layer by taking the log of the ratio of these two composites. 

In [44]:
# Define & visualize Workflows ImageCollection for before & after Sentinel-1 composites

#pre-disaster imagery
img = wf.ImageCollection.from_id("sentinel-2:L1C",start_datetime='2017-08-15', end_datetime='2017-09-15').pick_bands("red green blue")
img = img.filter(lambda img: img.properties["cloud_fraction"] <= 0.1) #filtering out the clouds
mosaic_pre = img.min(axis="images")  #axis='images')
mosaic_pre.visualize('before', map=m1) # scales=[(0, 1), (0, 1), (0, 1)],

#post disaster imagery
img2 = wf.ImageCollection.from_id("sentinel-2:L1C",start_datetime='2017-10-01', end_datetime='2017-11-01').pick_bands("red green blue")
img2 = img2.filter(lambda img: img.properties["cloud_fraction"] <= 0.06) #filtering out the clouds
mosaic_post = img2.min(axis="images")
mosaic_post.visualize('after', map=m1) # scales=[(0, 1), (0, 1), (0, 1)],

#take the log of the ratio of both composites (equivalent to difference of logs)
#calculate the ratio on the mosaic rather than the image collection, as visualise will only work with a single image
log_ratio = wf.log10(mosaic_pre / mosaic_post)

threshold =  0.15

# Threshold the log ratio layer  
change_detected = (log_ratio > threshold) 

# Morphological filtering
The log-ratio methodology is inherently noisy, so here we leverage the Workflows kernel capability to apply some simple morphological filtering to the change_d layer in order to clean up the deforestation detections.

In [45]:
# Define simple functions for erosion and dilation

def erode_op(map_layer, iters, kernel):
    map_layer = ~map_layer
    for i in range(iters):
        map_layer = wf.conv2d(map_layer, kernel) > 0
    map_layer = ~map_layer 
    return map_layer

def dilate_op(map_layer, iters, kernel):
    for i in range(iters):
        map_layer = map_layer * 1.0
        map_layer = wf.conv2d(map_layer, kernel) > 0
    return map_layer

# Define a kernel and perform one erosion followed by two dilations
kernel = wf.Kernel(dims=(3,3), data=[0., 1., 0.,
                                      1., 1., 1.,
                                      0., 1., 0.])

eroded = erode_op(change_detected, iters=1, kernel=kernel)
dilated = dilate_op(eroded, iters=2, kernel=kernel)

## Define custom ipywidget to visualize change detection mask 

The FullArray call leverages the supporting utilities to enable the user to draw a polygon around an area of interest to output the size of the areas where change was detected. 

In [46]:
# Place the center of the map in an interesting area
center = (18.466333, -66.105721)

# Initialize a workflow map
m = wf.map.map
m.center = center
m.zoom = 15
m.layout.width = '1200px'
m.layout.height = '900'



dc = DrawControl(circlemarker={}, rectangle={}, polyline={})
wf.map.add_control(dc)

log_ratio.visualize('change_detcted', map=m)

ipywidgets.HBox([wf.map])



`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jup…

In [47]:
import geojson
from geojson import Point, MultiPolygon, Polygon, Feature, FeatureCollection, dump
poly = Polygon([[settlements.geometry.x[0], settlements.geometry.y[0]], [settlements.geometry.x[0]+0.001, settlements.geometry.y[0]], [settlements.geometry.x[0]+0.01, settlements.geometry.y[0]+0.01], [settlements.geometry.x[0], settlements.geometry.y[0]+0.01], [settlements.geometry.x[0], settlements.geometry.y[0]]])
#poly = Polygon(([[settlements.geometry.x[0], settlements.geometry.y[0]], [settlements.geometry.x[0], settlements.geometry.y[0]], [settlements.geometry.x[0], settlements.geometry.y[0]], [settlements.geometry.x[0], settlements.geometry.y[0]], [settlements.geometry.x[0], settlements.geometry.y[0]]]))
features = []
features.append(Feature(properties={"Damage": settlements.grading[0]}, geometry=poly))
feature_collection = FeatureCollection(features)
with open('dominicaDamage.geojson', 'w') as f:
   dump(feature_collection, f)

NameError: name 'settlements' is not defined